Modules

In [1]:
from ete3 import Tree
import toytree
import toyplot
import toyplot.pdf
import itertools
import math
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import sys
pd.options.mode.chained_assignment = None

Functions

In [2]:
def min_div_to_chrom_assembly(df, target_sp, clevel_sps, tree):
    '''Returns the minimum distance between sp and a chromosome level assembly'''
    if target_sp in clevel_sps:
        return 0
    distances = [tree.get_distance(target_sp, sp) for sp in chrom_level]
    return min(distances)

def get_worst_species(df, decision_tree):
    '''Returns species ranking higher in decision tree'''
    for param, criteria in decision_tree.items():
        if criteria!="Max" and criteria!="Min":
            subd = df[df[param]!=criteria]
            if len(subd)==1:
                return subd.Species.values[0]
        else:
            if len(df[param].unique())==1:
                continue
            sorted_d = df.sort_values(by=param, ascending=False if criteria!="Max" else True)
            sorted_d.index = range(len(sorted_d))
            return sorted_d.Species.values[0]

def conservative_prune_dataset_by_pi(df, times_pi, decision_tree, tree):
    '''Thins dataset, ensuring species are at list times_pi*pi diverged and >2%
    keeping species according to decision tree'''
    
    exclude_species = []
    sp_universe = [phylo.get_leaf_names()]
    for sp1,sp2 in itertools.combinations(df.Species, 2):
        distance = tree.get_distance(sp1,sp2)
        pi1 = float(df[df.Species==sp1]["Pi_het"])
        pi2 = float(df[df.Species==sp2]["Pi_het"])
        max_pi = np.nanmax([pi1,pi2])
        min_distance = max_pi*times_pi
        if min_distance>=distance or distance<=0.02:
            worst_sp = get_worst_species(df[df.Species.isin([sp1,sp2])], decision_tree)
            exclude_species.append(worst_sp)
        
    pruned_df = df[~df.Species.isin(exclude_species)]
    return pruned_df
    
def prune_dataset_by_pi(df, times_pi, decision_tree, tree):
    '''Thins dataset, ensuring species are at list times_pi*pi diverged
    keeping species according to decision tree'''
    
    # All comparisons, record if pair is valid
    total_species = df.Species
    matrix = []
    for sp1,sp2 in itertools.combinations(total_species, 2):
        distance = tree.get_distance(sp1,sp2)
        pi1 = float(df[df.Species==sp1]["Pi_het"])
        pi2 = float(df[df.Species==sp2]["Pi_het"])
        max_pi = np.nanmax([pi1,pi2]) if not all(math.isnan(p) for p in [pi1,pi2]) else 10
        min_distance = max_pi*times_pi
        matrix.append([sp1,sp2,0 if min_distance>distance else 1, distance])
        
    c_df = pd.DataFrame(matrix)
    c_df.columns = ["sp1","sp2","acceptance","divergence"]
    return c_df
    # Find valid combination with highest numerber of species
    for n in range(len(total_species)+1)[::-1]:
        sys.stderr.write("Trying combination of {} species...\n".format(n))
        for sp_set in itertools.combinations(total_species, n):
            set_df = c_df[(c_df.sp1.isin(sp_set)) & 
                          (c_df.sp2.isin(sp_set))]
            validity = set_df.acceptance.sum()==len(set_df)
            if validity:
                sys.stderr.write("Found a valid set of {} species.\n".format(n))
                return sp_set

    sys.stderr.write("There's no safe dataset...!")

Read data

In [3]:
group_name = "Mammals"
meta = pd.read_csv("../data/{}_assembly_metadata.csv".format(group_name),sep=",")
phylo = Tree("./../trees/{}_phast.nh".format(group_name), format=1)

Minimum assembly stats and heterogametic sex filters

In [4]:
scafN50 = 350e3 if group_name in ["Birds","Mammals"] else 100e3
contigN50 = 25e3 if group_name in ["Birds","Mammals"] else 10e3
chrom_level_dist = 0.35 if group_name=="Birds" else 0.15
complex_system = "Complex_XY" if group_name=="Mammals" else "Complex_ZW"
heterogametic_sex = "male" if group_name=="Mammals" else "female"
heterogametic_sex = heterogametic_sex if group_name!="Snakes" else None

# Filtering scaffold assemblies for heterogametic sex
n = len(meta)
meta = meta[((meta.Sex!=heterogametic_sex) | (meta.AssemblyStatus=="Chromosome"))].reset_index(drop=True)
print("Filtered {} scaffold assemblies based on heterogametic sex".format(n-len(meta)))

# Filtering species in genus with complex system of sex determination 
n = len(meta)
meta = meta[(meta[complex_system]!=True)].reset_index(drop=True)
print("Filtered {} with complex system of sex determination".format(n-len(meta)))

# Filtering by assembly stats 
n = len(meta)
meta = meta[((meta.ScaffoldN50>scafN50) | (meta.AssemblyStatus=="Chromosome")) &
            (meta.ContigN50>contigN50)].reset_index(drop=True)
print("Filtered {} with stats below thershold".format(n-len(meta)))

Filtered 50 scaffold assemblies based on heterogametic sex
Filtered 9 with complex system of sex determination
Filtered 120 with stats below thershold


Thin by divergence, unless Aves, in which we take 1 species per Neoaves order (and focus in phylogenetic relationships that are well resolved (https://doi.org/10.1093/sysbio/syx041))

In [5]:
decision_tree = {"dnm":"Max",
                 "AssemblyStatus":"Chromosome",
                 "ScaffoldN50":"Max",
                 "Sex":heterogametic_sex
                }

if group_name!="Birds":
    thinned_df = conservative_prune_dataset_by_pi(meta, 15, decision_tree, phylo)

else:
    g1 = ["Columbiformes","Pterocliformes","Charadriiformes"]
    g2 = ["Otidiformes","Cuculiformes","Pelecaniformes"]
    g3 = ["Trogoniformes","Bucerotiformes","Coraciiformes"]
    g4 = ["Falconiformes","Passeriformes"]
    g5 = ["Accipitriformes","Gruiformes"]
    g6 = ["Anseriformes","Galliformes"]
    
    total = sorted(sum([g1,g2,g3,g4,g5,g6], []))
    order2group = {order:"Group{}".format(i+1) for i,g in enumerate([g1,g2,g3,g4,g5,g6]) for order in g}
    meta = meta[meta.Order.isin(total)].reset_index(drop=True)
    keep_species = []
    
    for order,df in meta.groupby("Order"):
        df.index = df.Species
        if order=="Passeriformes":
            keep_species.append(["Ficedula albicollis"])
        elif order in ["Anseriformes", "Galliformes"]:
            if order=="Galliformes":
                keep_species.append(list(df[df.Family!="Phasianidae"].index))
                keep_species.append(["Gallus_gallus"])
            else:
                keep_species.append(list(df.index))
        else:
            keep_species.append([df["ScaffoldN50"].idxmax()])
    
    keep_species = sum(keep_species,[])
    keep_species = ["_".join(sp.split()) for sp in keep_species]
    thinned_df = meta[meta.Species.isin(keep_species)]
    thinned_df["Group"] = [order2group[order] for order in thinned_df["Order"]]
    thinned_df = thinned_df.drop_duplicates("Species")
    thinned_df = conservative_prune_dataset_by_pi(thinned_df, 15, decision_tree, phylo)

Thin by distance to chromosome assembly

In [6]:
chrom_level = list(thinned_df[thinned_df.AssemblyStatus=="Chromosome"].Species)
thinned_df["min_div_clevel"] = [min_div_to_chrom_assembly(thinned_df, sp, chrom_level, phylo) if sp in phylo.get_leaf_names() else 1 for sp in thinned_df.Species]
thinned_df = thinned_df[thinned_df.min_div_clevel<=chrom_level_dist].reset_index(drop=True)

Output species list file and phylogeny

In [100]:
with open("../data/{}.txt".format(group_name),"w") as fh:
    fh.write(",".join(list(thinned_df.Species)) + "\n")
phylo.prune(thinned_df.Species, preserve_branch_length=True)
phylo.write(outfile="./../trees/{}.nwk".format(group_name),format=5)

if group_name=="Birds":
    for group,df in thinned_df.groupby("Group"):
        species = list(df.Species)
        # Neoaves
        if "Gallus_gallus" not in species:
            species.append("Gallus_gallus")
        # Outside branch
        else:
            pass
        # Species list
        fix_group = group.replace("Group","g")
        with open("./../data/Birds_{}.txt".format(fix_group), "w") as spf:
            spf.write(",".join(species) + "\n")
        # Tree
        phylo = Tree("./../trees/363-avian-2020-phast.nh", format=1)
        phylo.prune(species, preserve_branch_length=True)
        phylo.write(outfile="./../trees/Birds_{}.nwk".format(fix_group), format=5)

Birds subgroups phylogeny

In [10]:
phylo = Tree("./../trees/{}_phast.nh".format(group_name), format=1)
phylo.prune(thinned_df.Species.values, preserve_branch_length=True)

sp2group = thinned_df[["Species","Group"]].set_index("Species").to_dict()["Group"]
sp2order = thinned_df[["Species","Order"]].set_index("Species").to_dict()["Order"]
sp2common = thinned_df[["Species","Common name"]].set_index("Species").to_dict()["Common name"]

nwk = toytree.tree(phylo.write(), tree_format=1)
colors_tips = [toyplot.color.brewer.palette("Set2")[int(sp2group[sp][-1])] for sp in nwk.get_tip_labels()]
tips = ["{},{}".format(sp2order[sp],sp2common[sp]) for sp in nwk.get_tip_labels()]

canvas, axes, mark = nwk.draw(
    tip_labels_align =True,
    tip_labels = tips,
    tip_labels_colors = colors_tips,
    height = 500,
    tip_labels_style={
        "font-size": "8px",
    }
);

#toyplot.pdf.render(canvas, "pdfs/Bird_groupings.pdf")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="455.0px" height="500.0px" viewBox="0 0 455.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t99089e419c46400d90e60b2d8d4a9f3d"> Coraciiformes,Woodland Kingfisher Bucerotiformes,Northern Ground Hornbill Trogoniformes,Black-tailed Trogon Accipitriformes,Bald Eagle Falconiformes,Saker Falcon Passeriformes,Collared Flycatcher Pelecaniformes,Little Egret Gruiformes,Grey-winged Trumpeter Charadriiformes,Ruff Cuculiformes,Common Cuckoo Otidiformes,Red-crested Bustard Columbiformes,Domestic Pigeon (Rock Dove) Pterocliformes,Burchell's Sandgrouse Galliformes,Chicken (Red Junglefowl) Galliformes,White-crested Guan Anseriformes,Muscovy Duck Anseriformes,Magpie Goose